In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/saumya.nauni/Desktop/capstone/data/5-secs/tsne-data-5secs.csv')
df.head()

,identification_number,x,y
0,5-0,39.650500,23.295845
1,5-134,15.620601,-12.035807
2,5-138,25.467508,-11.551829
3,7-0,38.799675,22.477068
4,7-216,-26.743486,-4.428862


In [3]:
ann = pd.read_csv('/Users/saumya.nauni/Desktop/capstone/data/5-secs/vggish_5_sec_comb_labels.csv')
ann.head()

,vggish_point,label
0,100-0,calibration
1,100-314,humpback
2,100-315,humpback
3,100-316,humpback
4,1006-0,calibration


In [4]:
# Merge the two dataframes based on the vggish_point and identification_number column
merged = pd.merge(df, ann, how='left', left_on='identification_number', right_on='vggish_point')
merged_df = merged.drop(columns=['vggish_point'], axis = 1)
merged_df.head()

,identification_number,x,y,label
0,5-0,39.650500,23.295845,calibration
1,5-134,15.620601,-12.035807,mooring
2,5-138,25.467508,-11.551829,mooring
3,7-0,38.799675,22.477068,calibration
4,7-216,-26.743486,-4.428862,humpback


In [5]:
merged_df['label'].isna().sum()

100000

In [7]:
import plotly.graph_objs as go
from ipywidgets import VBox
import random

# Function to generate a list of random bright colors
def generate_bright_colors(num_colors):
    # Define a list of bright colors
    bright_colors = ["#FF5733", "#FFBD33", "#33FF57", "#336DFF", "#FF33D5", "#FF3333", "#33FFF6", "#B233FF", "#FFBE33", "#FF33A7", "#B3FF33", "#5733FF", "#33FFD4", "#B23333", "#33B3FF", "#FF8A33", "#6D33FF", "#33FFB6", "#FF336D", "#FFC833", "#33FF7F", "#33A7FF", "#FF33B6", "#33FF8A", "#33FF33", "#33B6FF", "#D533FF", "#33FF33", "#8A33FF", "#FF338A"]
    # Shuffle the list of bright colors to ensure each category gets a distinct color
    random.shuffle(bright_colors)
    return bright_colors[:num_colors]

# Get unique sounds and assign bright colors
unique_sounds = merged_df['label'].unique()
num_unique_sounds = len(unique_sounds)
bright_colors = generate_bright_colors(num_unique_sounds)

# Map sounds to their respective bright colors
sound_colors = dict(zip(unique_sounds, bright_colors))

# Replace NaN values with black
colors = merged_df['label'].map(sound_colors)
colors[merged_df['label'].isna()] = 'grey'

# Define size and opacity for each color
sizes = [2 if c == 'grey' else 5 for c in colors]
opacities = [0.4 if c == 'grey' else 1 for c in colors]

# Create legend entries
legend_entries = []
for sound, color in sound_colors.items():
    legend_entries.append(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(color=color),
        showlegend=True,
        name=sound
    ))

# Create main figure with scatter plot and legend entries
f = go.FigureWidget([go.Scatter(
    y=merged_df['y'],
    x=merged_df['x'],
    mode='markers',
    marker=dict(size=sizes, color=colors, opacity=opacities, line=dict(color='rgba(0,0,0,0)')),
    showlegend=False  # Don't show the legend for this trace
)] + legend_entries)  # Include legend entries in the main figure

# Update layout to display legend
f.update_layout(
    dragmode='lasso',
    autosize=False,
    width=1200,
    height=800,
    legend=dict(
        title='Legend',
        title_font_size=16,
        font=dict(size=12),  # Set the font size of the legend text
        traceorder='normal'  # Set the order of legend items
    )
)

def update_axes(xaxis, yaxis):
    scatter = f.data[0]
    scatter.x = merged_df[xaxis]
    scatter.y = merged_df[yaxis]
    with f.batch_update():
        f.layout.xaxis.title = xaxis
        f.layout.yaxis.title = yaxis

t = go.FigureWidget([go.Table(
    header=dict(values=['identification_number', 'label'],
                fill=dict(color='#C2D4FF'),
                align=['left'] * 5),
    cells=dict(values=[merged_df[col] for col in ['identification_number', 'label']],
               fill=dict(color='#F5F8FF'),
               align=['left'] * 5))])

def selection_fn(trace, points, selector):
    t.data[0].cells.values = [merged_df.loc[points.point_inds][col] for col in ['identification_number', 'label']]

f.data[0].on_selection(selection_fn)

# Display plot and table together
VBox((f, t))

/Users/saumya.nauni/.local/lib/python3.9/site-packages/jupyter_client/session.py:718: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



    'data': [{'marker': {'color': array(['#33FFF6', '#FFC833', '#FFC833', ..., '…